# Training the latent ODE model
<strong>From appendix D</strong>

To obtain the latent representation $z_{t_{0}}$, we traverse using RNN and obtain parameters of distribution $ q(\textbf{z}_{t_{0}} | \{ \textbf{x}_{t_{i}}, t_{i} \}_{i}, \theta_{enc})$. The algorithm is the following:


## Step 1
Run an RNN encoder through the time series and infer the parameters for the a posterior over $ \textbf{z}_{t_{0}}$:
$$ q(\textbf{z}_{t_{0}} | \{ \textbf{x}_{t_{i}}, t_{i} \}_{i}, \phi) = \mathcal{N}(\textbf{z}_{t_{0}} | \mu_{\textbf{z}_{t_{0}}}, \sigma_{\textbf{z}_{0}}) $$

where $\mu_{z_{0}}, \sigma_{z_{0}}$ comes from hidden state of $ RNN(\{ \textbf{x}_{t_{i}} , t_{i} \}_{i}, \phi) $

## Step 2
Sample $ \textbf{z}_{t_{0}} \sim q(\textbf{z}_{t_{0}} | \{ \textbf{x}_{t_{i}}, t_{i} \}_{i}) $

In [65]:
# import
import numpy as np
import pandas as pd
import autograd_extended
import tensorflow as tf
from keras.models import Model
from keras.layers import LSTM, Input, Lambda, Dense, RepeatVector
from keras import backend as K
from keras import objectives

# obtain simple time series with sin function
df = pd.read_csv('./data.csv')

# split data into x_train, x_val, y_train, y_val
ratio = 0.75
idx = int(len(df) * ratio)
df[1:20]["x"]
x_train, x_test, y_train, y_test = df[:idx]["x"], df[idx:]["x"], df[:idx]["y"], df[:idx]["y"]

Define and run the autoencoder model

In [66]:
# Run an RNN encoder on the time series

# Params
timesteps = 100
input_dim = 1
lstm_dim = 32
latent_dim = 100
batch_size = 1
x_train.reshape = (-1, timesteps, 1)  # we make 100 samples
y_train.reshape = (-1, timesteps, 1)
x_test.reshape = (-1, timesteps, 1)
y_test.reshape = (-1, timesteps, 1)


# Encoder model
def sample_z(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


# Input
inputs = Input(shape=(timesteps, input_dim,), name='encoder_input')

# LSTM encoding
h = LSTM(lstm_dim)(inputs)

# VAE Z layer
z_mean = Dense(latent_dim)(h)
z_log_sigma = Dense(latent_dim)(h)

# Obtain z
z = Lambda(sample_z, output_shape=(latent_dim,))([z_mean, z_log_sigma])

encoder = Model(inputs, z_mean, name="encoder")

# Decoder model
decoder_h = LSTM(lstm_dim, return_sequences=True)
decoder_mean = LSTM(input_dim, return_sequences=True)

h_decoded = RepeatVector(timesteps)(z)
h_decoded = decoder_h(h_decoded)

# decoded layer
outputs = decoder_mean(h_decoded)

# decoder, from latent space to reconstructed inputs
decoder_inputs = Input(shape=(latent_dim,))

_h_decoded = RepeatVector(timesteps)(decoder_inputs)
_h_decoded = decoder_h(_h_decoded)

decoder_outputs = decoder_mean(_h_decoded)

decoder = Model(decoder_inputs, decoder_outputs)

# Build vae
vae = Model(inputs, outputs, name='vae_lstm')
def vae_loss(x, x_decoded_mean):
    xent_loss = objectives.mse(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
    loss = xent_loss + kl_loss
    return loss

vae.compile(optimizer='adam', loss=vae_loss)

encoder.summary()
decoder.summary()
vae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 100, 1)            0         
_________________________________________________________________
lstm_45 (LSTM)               (None, 32)                4352      
_________________________________________________________________
dense_51 (Dense)             (None, 100)               3300      
Total params: 7,652
Trainable params: 7,652
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 100)               0         
_________________________________________________________________
repeat_vector_8 (RepeatVecto (None, 100, 100)          0         
_________________________________________________________________
lstm_46 

In [67]:
# Train the VAE to obtain z



## Step 3